# ML Experiment-7

In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 38.3 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
# Structure of the Bayesian Network
model_structure = DiscreteBayesianNetwork(
    [
        ('Burglary', 'Alarm'),
        ('Earthquake', 'Alarm'),
        ('Alarm', 'JohnCalls'),
        ('Alarm', 'MaryCalls')
    ]
)

In [ ]:
#CPDs (Conditional Probability Distributions)
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.999], [0.001]])
"""
The first row corresponds to Burglary = 0 (i.e., no burglary occurred).
The second row corresponds to Burglary = 1 (i.e., burglary occurred).
"""
print("Burglary:\n",cpd_burglary)

cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.998], [0.002]])
print("Earthquake:\n",cpd_earthquake)

Burglary:
 +-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+
Earthquake:
 +---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_alarm = TabularCPD(
    variable='Alarm',
    variable_card=2,
    values=[
        [0.999, 0.71, 0.06, 0.05],      # P(Alarm=0|Burglary, Earthquake)
        [0.001, 0.29, 0.94, 0.95]       # P(Alarm=1|Burglary, Earthquake)
    ],
    evidence=['Burglary', 'Earthquake'],
    evidence_card=[2, 2]
)
print("Alarm:\n",cpd_alarm)

cpd_johncalls = TabularCPD(
    variable='JohnCalls',
    variable_card=2,
    values=[
        [0.95, 0.1],
        [0.05, 0.9]
    ],
    evidence=['Alarm'],
    evidence_card=[2]
)
print("JohnCalls:\n",cpd_johncalls)

cpd_marycalls = TabularCPD(
    variable='MaryCalls',
    variable_card=2,
    values=[
        [0.99, 0.3],
        [0.01, 0.7]
    ],
    evidence=['Alarm'],
    evidence_card=[2]
)
print("MaryCalls:\n",cpd_marycalls)

Alarm:
 +------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+
JohnCalls:
 +--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+--------

In [ ]:
model_structure.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls)
model_structure.check_model()

True

In [ ]:
inference = VariableElimination(model_structure)
query_result = inference.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1})
print(query_result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


## Query-1: P(JohnCall, MaryCall, Alarm|Burglary=0, Earthquake=0)

In [ ]:
evidence = {'Burglary': 0, 'Earthquake': 0}
result1 = inference.query(variables=['JohnCalls', 'MaryCalls', 'Alarm'], evidence=evidence)
print(result1)

+--------------+--------------+----------+----------------------------------+
| JohnCalls    | MaryCalls    | Alarm    |   phi(JohnCalls,MaryCalls,Alarm) |
+==============+==============+==========+==================================+
| JohnCalls(0) | MaryCalls(0) | Alarm(0) |                           0.9396 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(0) | Alarm(1) |                           0.0000 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(0) |                           0.0095 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(1) |                           0.0001 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(1) | MaryCalls(0) | Alarm(0) |                           0.0495 |
+--------------+--------------+----------+----------------------

In [ ]:
probability1 = result1.values[1][1][1]
print("Probability of Alarm Ringing when JohnCalls=True, MaryCalls=True, Burglary=False, Earthquake=False: ", probability1)

Probability of Alarm Ringing when JohnCalls=True, MaryCalls=True, Burglary=False, Earthquake=False:  0.0006300000000000001


## Query-2: P(JohnCall)

In [ ]:
result2 = inference.query(variables=['JohnCalls'])
print(result2)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
probability2 = result2.values[1]
print("Probability of JohnCall: ", probability2)

Probability of JohnCall:  0.0521389757


## Query-3: P(Burglary|JohnCall, MaryCall)

In [ ]:
evidence = {
    'JohnCalls': 1,
    'MaryCalls': 1
}
result3 = inference.query(variables=['Burglary'], evidence=evidence)
print(result3)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
probability3 = result3.values[1]
print("Probability of Burglary when JohnCalls=True, MaryCalls=True: ", probability3)

Probability of Burglary when JohnCalls=True, MaryCalls=True:  0.284171835364393


## Query-4: P(Earthquake|JohnCalls=True, MaryCalls=False)

In [ ]:
evidence = {
    'JohnCalls': 1,
    'MaryCalls': 0
}
result4 = inference.query(variables=['Earthquake'], evidence=evidence)
print(result4)

+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9955 |
+---------------+-------------------+
| Earthquake(1) |            0.0045 |
+---------------+-------------------+


In [ ]:
probability4 = result4.values[1]
print("Probability of Earthquake when JohnCalls=True, MaryCalls=False: ", probability4)

Probability of Earthquake when JohnCalls=True, MaryCalls=False:  0.0045386400007529125


## Query-5: P(Alarm|Burglary=True, Earthquake=False)

In [ ]:
evidence = {
    'Burglary': 1,
    'Earthquake': 0
}
result5 = inference.query(variables=['Alarm'], evidence=evidence)
print(result5)

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.0600 |
+----------+--------------+
| Alarm(1) |       0.9400 |
+----------+--------------+


In [ ]:
probability5 = result5.values[1]
print("Probability of Alarm when Burglary=True, Earthquake=False: ", probability5)

Probability of Alarm when Burglary=True, Earthquake=False:  0.94


## Query-6: P(JohnCalls|Alarm=True, Burglary=False, Earthquake=True)

In [ ]:
evidence = {
    'Alarm': 1,
    'Burglary': 0,
    'Earthquake': 1
}
result6 = inference.query(variables=['JohnCalls'], evidence=evidence)
print(result6)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.1000 |
+--------------+------------------+
| JohnCalls(1) |           0.9000 |
+--------------+------------------+


In [ ]:
probability6 = result6.values[1]
print("Probability of JohnCalls when Alarm=True, Burglay=False, Earthquake=True: ", probability6)

Probability of JohnCalls when Alarm=True, Burglay=False, Earthquake=True:  0.9
